In [2]:
%load_ext autoreload
%autoreload 2
import scalable_gps
from scalable_gps.kernels import Matern32Kernel

import jax.numpy as jnp
import gpytorch as gpt
import numpy as np
import torch
import gpflux

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import jax.random as jr

# kernel_gpt = gpt.kernels.MaternKernel(nu=1.5)
kernel_gpt = gpt.kernels.ScaleKernel(gpt.kernels.MaternKernel(nu=1.5))


data = jr.normal(key=jr.PRNGKey(0), shape=(10, 1))

data_torch = torch.from_numpy(np.array(data)).float()
data_jax = jnp.array(data)

covar_gpt = kernel_gpt(data_torch, data_torch)

print('Gpytorch hparams : ', kernel_gpt.lengthscale)
# prnt(kernel_gpt.outputscale)
kernel_jax = Matern32Kernel(kernel_config={"signal_scale": np.sqrt(np.log(2)), "length_scale": np.log(2)})

covar = kernel_jax.K(data_jax, data_jax)
print(covar_gpt.shape)
print(covar.shape)

# print(np.linalg.norm(covar_gpt.numpy() - covar))

M = 500000

rff = kernel_jax.Phi(key=jr.PRNGKey(0), n_features=M, x=data_jax, recompute=True)

covar_rff = rff @ rff.T
print(np.linalg.norm(covar_gpt.numpy() - covar))

print(np.linalg.norm(covar_rff - covar))


Gpytorch hparams :  None
torch.Size([10, 10])
(10, 10)
6.8177415e-06
0.0097866105


In [14]:
from gpflux.layers.basis_functions.fourier_features import RandomFourierFeaturesCosine
from gpflux.sampling.kernel_with_feature_decomposition import KernelWithFeatureDecomposition
from gpflow.kernels import RBF, Matern32
import tensorflow as tf

M = 1000
feature_functions = RandomFourierFeaturesCosine(
    kernel=Matern32(lengthscales=1.),
    n_components=M,
    dtype=tf.float32,
)

feature_coefficients = tf.ones((M, 1), dtype=tf.float32)
kernel = KernelWithFeatureDecomposition(
    kernel=None, feature_functions=feature_functions, feature_coefficients=feature_coefficients
)

print(kernel(tf.convert_to_tensor(data_jax, dtype=tf.float32), tf.convert_to_tensor(data_jax, dtype=tf.float32)))


TypeError: Exception encountered when calling layer "random_fourier_features_cosine_10" "                 f"(type RandomFourierFeaturesCosine).

`x` and `y` must have the same dtype, got tf.float32 != tf.float64.

Call arguments received by layer "random_fourier_features_cosine_10" "                 f"(type RandomFourierFeaturesCosine):
  • inputs=tf.Tensor(shape=(10, 1), dtype=float32)